<a href="https://colab.research.google.com/github/Alina-89/Academic_RAG/blob/main/Academic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio pymupdf sentence-transformers
!pip install chromadb
!pip install torch transformers
!pip install nbstripout


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!nbstripout --extra-keys "metadata.widgets" Academic_RAG.ipynb

Could not strip 'Academic_RAG.ipynb': file not found


In [1]:
!ls -l


total 4
drwxr-xr-x 1 root root 4096 Jun  9 16:44 sample_data


In [6]:
!pwd
!ls -l


/content
total 392
-rw-r--r-- 1 root root 393668 Jun  9 16:53 Academic_RAG.ipynb
drwxr-xr-x 1 root root   4096 Jun  9 16:44 sample_data


In [9]:
!nbstripout --extra-keys "metadata.widgets" Academic_RAG.ipynb


In [3]:
!ls -1

sample_data


In [ ]:
import os
import gradio as gr
import fitz
from sentence_transformers import SentenceTransformer
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch





#Extract the text from the PDFs

In [ ]:
#Function that extracts text from the pdf
def extract_text(pdf_file):
  doc=fitz.open(stream=pdf_file, filetype="pdf")
  text=""
  for page in doc:
    text+=page.get_text()
  return text

#Split the text into sentences

In [ ]:
#Split text into sentences
def naive_sent_tokenize(text):
    # Splits on ., ?, or ! followed by a space or end of string
    sentences = re.split(r'(?<=[.!?]) +', text)
    return sentences


# Combine sentences into chunks of max. 100 tokens


In [ ]:

#Chunk the sentences

def chunk_text_by_sentences(text, max_tokens=100):
    sentences = naive_sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_length = 0
    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length > max_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_length = sentence_length
        else:
            current_chunk += " " + sentence
            current_length += sentence_length
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks





#Import Chroma and create a collection

In [ ]:
#Import Chroma and create a collection

import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(
    persist_directory="chroma_db",  # where to store your DB files
    anonymized_telemetry=False
))

# Create or get your collection
collection = client.get_or_create_collection(name="pdf_chunks")

#Load the embedding model and embed the chunks

In [ ]:
#Load the embedding model

embedding_model=SentenceTransformer('all-MiniLM-L6-v2')

#Embed the text chunks
def embed_text(text, file_name="document.pdf"):
    chunks = chunk_text_by_sentences(text)
    embeddings = embedding_model.encode(chunks)
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        collection.add(
            documents=[chunk],
            embeddings=[embedding],
            ids=[f"{file_name}_{i}"],
            metadatas=[{"chunk_index": i, "source": file_name}]
        )

    return f"Stored {len(chunks)} chunks in Chroma for {file_name}"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Load the LLM locally


In [ ]:
# Load Phi-2 model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): LayerNorm((2560,), eps=1

#User enters a query, the query is embedded, the prompt (style+user query+retrieved chunks) is sent to the model, and the model will generate an answer.

In [ ]:
#Lets user enter a query, embedds the query and retrieves relevant chunks then the model generates based on query and chunks and also on style
def chat_with_pdf(user_query, style, top_k=5):
    if not user_query.strip():
        return "Please enter a question."

    # Embed the query
    query_embedding = embedding_model.encode([user_query])[0]

    # Search in ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    if not results["documents"]:
        return "No relevant chunks found."

    # Prepare prompt
    chunks = results["documents"][0]
    prompt_style = {
        "Academic": "Answer the question in an academic tone.",
        "Friendly": "Answer the question like you are someone's best friend.",
        "Explain like I am 5": "Answer the question in a simple way, like you're explaining to a 5 year old child."
    }

    prompt = f"{prompt_style.get(style, '')}\n\nContext:\n" + "\n\n".join(chunks) + f"\n\nQuestion: {user_query}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()


In [ ]:
#Interface with gradio
with gr.Blocks(css="""
/* 🌙 Dark Mode Background */
.gradio-container {
    background-color: #18181B;
    color: #FFFFFF;
    font-family: 'Poppins', sans-serif;
    padding: 20px;
    display: flex;
    justify-content: center;
}

/* Stack content vertically and center */
#main-column {
    display: flex;
    flex-direction: column;
    align-items: center;
    max-width: 800px;
    width: 100%;
    margin: auto;
}

/* 🖼️ Image Styling */
.gr-image {
    border-radius: 12px;
    box-shadow: 4px 4px 10px rgba(255, 255, 255, 0.2);
}

/* ✏️ Textbox Enhancements */
.gr-textbox {
    width: 90%;
    font-size: 18px;
    padding: 10px;
    border: 2px solid #4A4A4D;
}

/* 🎨 Button Customization */
.gr-button {
    background-color: #5A67D8;
    color: pink;
    font-size: 16px;
    padding: 12px 18px;
    border-radius: 8px;
    transition: 0.2s ease-in-out;
}

/* ✨ Refine Labels */
label {
    font-weight: bold;
    color: #D1D5DB;
}
""") as demo:
    with gr.Column(elem_id="main-column"):
        gr.Markdown("<h2 style='color: #EAB308;'>📄 Upload your PDF</h2><p style='color: #9CA3AF;'>Then ask questions about it</p>")

        file_input = gr.File(label="Upload a file", type="binary")
        style_input = gr.Radio(["Academic", "Friendly", "Explain like I am 5"], label="Choose a style")
        #upload_btn = gr.Button("Upload PDF")

        gr.Markdown("<h2 style='color: #34D399;'>💬 Ask a question</h2>")
        query_input = gr.Textbox(label="Your question")
        query_btn = gr.Button("Ask")
        query_output = gr.Textbox(label="Answer")

        query_btn.click(fn=chat_with_pdf,
                        inputs=[query_input, style_input],
                        outputs=[query_output])

    demo.launch(debug=True)



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ad6c991f1390963812.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ad6c991f1390963812.gradio.live
